

## Simple Linear Regression


## Learning Goals

* Understand array reshaping
* Review how to make plots
* Feel comfortable with simple linear regression

In [2]:
# import the necessary libraries
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns

## Simple Linear Regression
Linear regression and its many extensions are a workhorse of the statistics and data science community, both in application and as a reference point for other models. Thus, this tutorial will introduce you to building and fitting linear regression models and some of the process behind it, so that you can 1) fit models to data you encounter 2) experiment with different kinds of linear regression and observe their effects 3) see some of the technology that makes regression models work.


### Linear regression with a toy dataset
We first examine a toy problem, focusing our efforts on fitting a linear model to a small dataset with three observations.  Each observation consists of one predictor $x_i$ and one response $y_i$ for $i = 1, 2, 3$,

\begin{align*}
(x , y) = \{(x_1, y_1), (x_2, y_2), (x_3, y_3)\}.
\end{align*}

To be very concrete, let's set the values of the predictors and responses.

\begin{equation*}
(x , y) = \{(1, 2), (2, 2), (3, 4)\}
\end{equation*}

There is no line of the form $\beta_0 + \beta_1 x = y$ that passes through all three observations, since the data are not collinear. Thus our aim is to find the line that best fits these observations in the *least-squares sense*, as discussed in lecture.

<div class="exercise"><b>Exercise (3 min)</b></div>

* Make two numpy arrays out of this data, x_train and y_train
* Make points into a very simple scatterplot

In [5]:
# your code here
# x_train = ?
# y_train =?

In [6]:
# Make a simple scatterplot



#### Formulae
Linear regression is special among the models we study beuase it can be solved explicitly. While most other models (and even some advanced versions of linear regression) must be solved itteratively, linear regression has a formula where you can simply plug in the data.

For the single predictor case it is:
    \begin{align}
      \beta_1 &= \frac{\sum_{i=1}^n{(x_i-\bar{x})(y_i-\bar{y})}}{\sum_{i=1}^n{(x_i-\bar{x})^2}}\\
      \beta_0 &= \bar{y} - \beta_1\bar{x}\
    \end{align}
    
Where $\bar{y}$ and $\bar{x}$ are the mean of the y values and the mean of the x values, respectively.

From the re-aranged second equation we can see that the best-fit line  passes through $(\bar{x},\bar{y})$, the center of mass of the data

From any of the first equations, we can see that the slope of the line has to do with whether or not an x value that is above/below the center of mass is typically paired with a y value that is likewise above/below, or typically paired with one that is opposite.

###  Building a model from scratch
In this part, we will solve the equations for simple linear regression and find the best fit solution to our toy problem.

The snippets of code below implement the linear regression equations on the observed predictors and responses, which we'll call the training data set.  Let's walk through the code.

### generate x_train and y_train for your linear regression

In [62]:
x_train = np.array([1,2,3])
y_train = np.array([2,3,6])

In [63]:
x_train

array([1, 2, 3])

In [7]:
# first, compute means
y_bar = np.mean(y_train)
x_bar = np.mean(x_train)

# build the two terms
numerator = np.sum( (x_train - x_bar)*(y_train - y_bar) )

denominator = np.sum((x_train - x_bar)**2)


In [13]:
#slope beta1
beta_1 = numerator/denominator

#intercept beta0
beta_0 = y_bar - beta_1*x_bar

print("The best-fit line is {0:3.2f} + {1:3.2f} * x".format(beta_0, beta_1))
print(f'The best fit is {beta_0}')

The best-fit line is -0.33 + 2.00 * x
The best fit is -0.3333333333333335


### Building a model with `statsmodels` and `sklearn`

Now that we can concretely fit the training data from scratch, let's learn two `python` packages to do it all for us:
* [statsmodels](http://www.statsmodels.org/stable/regression.html) and 
* [scikit-learn (sklearn)](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

Our goal  is to show how to implement simple linear regression with these packages.  For an important sanity check, we compare the $\beta$ values from `statsmodels` and `sklearn` to the $\beta$ values that we found from above with our own implementation.

For the purposes of this lab, `statsmodels` and `sklearn` do the same thing.  More generally though, `statsmodels` tends to be easier for inference \[finding the values of the slope and intercept and dicussing uncertainty in those values\], whereas `sklearn` has machine-learning algorithms and is better for prediction \[guessing y values for a given x value\]. (Note that both packages make the same guesses, it's just a question of which activity they provide more support for.

**Note:** `statsmodels` and `sklearn` are different packages!  Unless we specify otherwise, you can use either one.

In [14]:
import statsmodels.api as sm

####  sm.OLS(y_train, x_train)

In [15]:
# build the OLS model (ordinary least squares) from the training data
testmodel_toy_ms = sm.OLS(y_train, x_train)

In [16]:
results_testmodel_toy_ms = testmodel_toy_ms.fit()


In [17]:
import warnings
warnings.filterwarnings('ignore')
print(results_testmodel_toy_ms.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.985
Model:                            OLS   Adj. R-squared (uncentered):              0.978
Method:                 Least Squares   F-statistic:                              135.2
Date:                Tue, 19 Mar 2024   Prob (F-statistic):                     0.00732
Time:                        18:42:32   Log-Likelihood:                         -2.1042
No. Observations:                   3   AIC:                                      6.208
Df Residuals:                       2   BIC:                                      5.307
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

####  Only one coefficient, Missing the y-intercept term! 

In [18]:
# create the X matrix by appending a column of ones to x_train
X = sm.add_constant(x_train)

#  an intercept term (also known as a bias or constant term) is often included.
# The sm.add_constant() is used to add this intercept term to your input features

In [19]:
print(x_train)

[[1]
 [2]
 [3]]


In [20]:
print(X)

[[1. 1.]
 [1. 2.]
 [1. 3.]]


In [21]:
# build the OLS model (ordinary least squares) from the training data

x_train = np.array([1,2,3])
y_train = np.array([2,3,6])

X = sm.add_constant(x_train)

toyregr_sm = sm.OLS(y_train, X)

# do the fit and save regression info (parameters, etc) in results_sm
results_sm = toyregr_sm.fit()

In [22]:

print(results_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     12.00
Date:                Tue, 19 Mar 2024   Prob (F-statistic):              0.179
Time:                        18:42:37   Log-Likelihood:                -2.0007
No. Observations:                   3   AIC:                             8.001
Df Residuals:                       1   BIC:                             6.199
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3333      1.247     -0.267      0.8

In [23]:
# pull the beta parameters out from results_sm
beta0_sm = results_sm.params[0]
beta1_sm = results_sm.params[1]

print("The regression coefficients from the statsmodels package are: beta_0 = {0:8.6f} and beta_1 = {1:8.6f}".format(beta0_sm, beta1_sm))

The regression coefficients from the statsmodels package are: beta_0 = -0.333333 and beta_1 = 2.000000


### Now let's turn our attention to the `sklearn` library.

In [27]:
from sklearn import linear_model

In [28]:
# build the least squares model
toyregr = linear_model.LinearRegression()
results = toyregr.fit(x_train, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[1 2 3].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

#### New concept in Python,"Shape"
In the context of NumPy arrays or data structures in scientific computing, "shape" refers to the dimensions or structure of a multidimensional array. The shape defines how many elements there are along each axis (dimension) of the array.

Here's a breakdown of what "shape" means for arrays with different numbers of dimensions:

1D array (vector): The shape is a single number representing the length of the vector. For example, a shape of (n,) means the array has n elements.

2D array (matrix): The shape includes two numbers representing the number of rows and columns, respectively. 

#### A shape of (m, n) means the array has m rows and n columns.

`Scikit-learn` is the main `python` machine learning library. It consists of many learners which can learn models from data, as well as a lot of utility functions such as `train_test_split`. It can be used in `python` by the incantation `import sklearn`.

In scikit-learn, an **estimator** is a Python object that implements the methods fit(X, y) and predict(T)

Let's see the structure of `scikit-learn` needed to make these fits. `.fit` always takes two arguments:
```python
  estimator.fit(Xtrain, ytrain)
```

Critically, `Xtrain` must be in the form of an *array of arrays* (or a 2x2 array) with the inner arrays each corresponding to one sample, and whose elements correspond to the feature values for that sample (visuals coming in a moment).

`ytrain` on the other hand is a simple array of responses.  These are continuous for regression problems.

![](images/sklearn2.jpg)

In [29]:
# 1D array 
array_1d = np.array([1, 2, 3, 4, 5])
print(array_1d.shape) 

(5,)


In [30]:
# 2D array of shape (2, 3)
array_2d = np.array([[1, 2, 3], [4, 5, 6]])
print(array_2d)
print(array_2d.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)


In [31]:
# Suppose we have an array with 12 elements
original_array = np.arange(12)
print("Original array:", original_array)

Original array: [ 0  1  2  3  4  5  6  7  8  9 10 11]


The reshape method in NumPy is used to change the shape of an array without changing its data. 

Essentially, it lets you rearrange the array's elements to have a specified shape, as long as the new shape is compatible with the total number of elements in the original array.

In [32]:
# Reshape it to a 3x4 array
reshaped_array = original_array.reshape((3, 4))
print("Reshaped array (3x4):\n", reshaped_array)

Reshaped array (3x4):
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


#### Here are some key points about reshape:

The new shape must be compatible with the total number of elements. In the example above, the original array has 12 elements, so the new shape must also be able to accommodate exactly 12 elements (e.g., 3x4, 2x6, 1x12, etc.).

You can pass -1 as one of the dimensions, and NumPy will automatically infer the correct dimension size that will keep the total number of elements the same.

In [33]:
reshaped_array = original_array.reshape((2, -1))
reshaped_array

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

<div class="exercise"><b>Exercise (5 min)</b></div>
* make an array with shape (2,3)
* reshape it to a size that you want

In [34]:
# your code here

In [40]:
toyregr = linear_model.LinearRegression()

# Reshape to be a proper 2D array

x_train = x_train.reshape(3,1)
y_train = y_train.reshape(3,1)

#x_train = x_train.reshape(x_train.shape[0], 1)
#y_train = y_train.reshape(y_train.shape[0], 1)

#print(x_train.shape)
# save regression info (parameters, etc) in results_skl
results = toyregr.fit(x_train, y_train)

# pull the beta parameters out from results_skl
beta0_skl = toyregr.intercept_
beta1_skl = toyregr.coef_[0]
print(beta0_skl)
print(beta1_skl)

[-0.33333333]
[2.]


In [44]:
predicted_y = toyregr.predict(x_train)

r2 = toyregr.score(x_train, y_train)
r2 

0.9230769230769231

In [50]:
from sklearn.metrics import mean_squared_error

In [49]:
mean_squared_error(y_train, toyregr.predict(x_train))

0.22222222222222218